In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
pd.set_option('display.max_rows', 20)
train=pd.read_csv("/Users/sunqiuhan/Desktop/train.csv")
train=train.fillna(0)
test=pd.read_csv("/Users/sunqiuhan/Desktop/test.csv")
test=test.fillna(0)

In [2]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
# Initialize the constructor
model = Sequential()

# Add an input layer 
model.add(Dense(180, activation='relu', input_dim=20))

# Add one hidden layer 
model.add(Dense(140, activation='relu'))

# Add one hidden layer 
model.add(Dense(80, activation='relu'))

# Add one hidden layer
model.add(Dense(40, activation='relu'))

# Add an output layer 
model.add(Dense(1, activation='sigmoid'))

In [4]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [5]:
# x variables need to exclude reordered
x_train = np.array(train.drop('reordered',axis=1))
y_train = np.array(pd.DataFrame(train["reordered"]))
x_scaled = preprocessing.scale(x_train)

In [15]:
model.fit(x_scaled, y_train,validation_split=0.33,epochs=200,verbose=1)

Train on 762998 samples, validate on 375805 samples
Epoch 1/50
762998/762998 [==============================] - 96s 126us/step - loss: 0.2470 - acc: 0.9099 - val_loss: 0.2521 - val_acc: 0.9085
Epoch 2/50
 79136/762998 [==>...........................] - ETA: 1:17 - loss: 0.2481 - acc: 0.9092

KeyboardInterrupt: 

In [7]:
# calculate prediction accuracy
x_test = np.array(test.drop('reordered',axis=1))
y_test = np.array(pd.DataFrame(test["reordered"]))
x_tscaled = preprocessing.scale(x_test)

In [8]:
y_pred = model.predict_proba(x_tscaled)

490442/490442 [==============================] - 17s 34us/step


In [9]:
test["prediction"]=y_pred
def label_pred (row):
    if row['prediction']>0.3:
        return 1
    else:
        return 0
test['pred'] = test.apply (lambda row: label_pred (row),axis=1)

In [10]:
tp=len(test[(test["pred"]==1)&(test["reordered"]==1)])

In [11]:
fn=len(test[(test["pred"]==0)&(test["reordered"]==1)])

In [12]:
fp=len(test[(test["pred"]==1)&(test["reordered"]==0)])

In [13]:
tn=len(test[(test["pred"]==0)&(test["reordered"]==0)])

In [16]:
# produce the confusion matrix
d = {'actual=0': [tn, fp], 'actual=1': [fn, tp]}
df = pd.DataFrame(data=d,index=['prediction=0','prediction=1'])

In [17]:
df

,actual=0,actual=1
prediction=0,425052,31099
prediction=1,17685,16606


In [33]:
score = model.evaluate(x_test, y_test,verbose=1)
print(score)

492878/492878 [==============================] - 17s 35us/step
[1.5514809210515239, 0.90374291406798435]


In [10]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score

# Confusion matrix
confusion_matrix(y_test, y_pred)

ValueError: Can't handle mix of binary and continuous

In [11]:
precision_score(y_test, y_pred)

ValueError: Can't handle mix of binary and continuous

In [ ]:
# try to figure out which variable is important to the model. For example, here, I am trying to
# see how the model's accuray changes when the variable "user_product.orders" is not included
x_train = np.array(train.drop(['reordered','user_product.orders'],axis=1))
x_scaled = preprocessing.scale(x_train)
model.fit(x_scaled, y_train,validation_split=0.33,epochs=200,verbose=1)
x_test = np.array(test.drop(['reordered','user_product.orders'],axis=1))
x_tscaled = preprocessing.scale(x_test)
y_pred = model.predict_proba(x_tscaled)
test["prediction"]=y_pred
def label_pred (row):
    if row['prediction']>0.3:
        return 1
    else:
        return 0
test['pred'] = test.apply (lambda row: label_pred (row),axis=1)